In [1]:
#import dependencies
import pandas as pd
import requests 
from pprint import pprint

In [3]:
#import csv of oscar winners
oscar_movie_db=pd.read_csv("../DataSources/Oscars1934-2015.csv")
pd.to_numeric(oscar_movie_db.Year, errors='coerce')

#Ensure that all the "Names" in the oscar_movie_db are formated correctly (no spaces at end of name)
for i,row in oscar_movie_db.iterrows():    
    j = oscar_movie_db['Name'].iloc[i]
    if j[-1] != " ":
        pass
    else:
        j=j[:-1]
        oscar_movie_db['Name'].iloc[i]=j

#create a dataframe of best picture winners and nominees from 1934-2015.
df_movies = oscar_movie_db[((oscar_movie_db['Award']== 'Best Picture')|(oscar_movie_db['Award']== 'Best Motion Picture'))]
df_movies=df_movies.reset_index(drop=True)
movies= list(df_movies['Name'])


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
df_movies.head()

,Year,Ceremony,Award,Winner,Name,Film
0,1944,17,Best Motion Picture,0,Double Indemnity,Paramount
1,1944,17,Best Motion Picture,0,Gaslight,Metro-Goldwyn-Mayer
2,1944,17,Best Motion Picture,1,Going My Way,Paramount
3,1944,17,Best Motion Picture,0,Since You Went Away,Selznick International Pictures
4,1944,17,Best Motion Picture,0,Wilson,20th Century-Fox


In [5]:
#TMDB API CALLs to get movie id (to be able to make a more specific API call later)
movie_ids = []
base_url1='https://api.tmdb.org/3/search/movie'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

#for all movies in best picture df, run API call by title and year and extract movie title and movie id
for i in range (len(df_movies)):
    combined_url1=base_url1+api_key+'&query='+df_movies['Name'][i]+'&year='+str(df_movies['Year'][i])
    try:
        movieData_TMDB=requests.get(combined_url1).json()
        movie_id=movieData_TMDB['results'][0]['id']
        movie_title=movieData_TMDB['results'][0]['original_title']
        release_date=movieData_TMDB['results'][0]['release_date']
        
        movie_ids.append([movie_title, release_date, movie_id])
    except:
        pass

movie_ids_df=pd.DataFrame(movie_ids)
movie_ids_df.columns=['Title', 'Release Date', 'Movie ID']
movie_ids_df.head()


,Title,Release Date,Movie ID
0,Double Indemnity,1944-04-24,996
1,Gaslight,1944-05-04,13528
2,Going My Way,1944-05-15,17661
3,Since You Went Away,1944-07-20,42401
4,Wilson,1944-08-01,84084


In [6]:
#Run API API calls by movie id for more details
tmdb_predf=[]
base_url2 = 'https://api.tmdb.org/3/movie/'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

for i in range(len(movie_ids_df)):
    combined_url2=base_url2+ str(movie_ids_df['Movie ID'][i])+ api_key 
    try:
        movieData_TMDB=requests.get(combined_url2).json()
        movie_title=movieData_TMDB['title']
        budget=movieData_TMDB['budget']
        popularity=movieData_TMDB['popularity']
        release_date=movieData_TMDB['release_date']
        revenue=movieData_TMDB['revenue']
        runtime=movieData_TMDB['runtime']
        vote_average=movieData_TMDB['vote_average']
        vote_count=movieData_TMDB['vote_count']
        
        tmdb_predf.append([movie_title, budget, popularity, release_date, revenue, runtime, vote_average, vote_count])
    except:
        pass
    
theMoviesDF_TMDB=pd.DataFrame(tmdb_predf)
theMoviesDF_TMDB.columns=['Title', 'Budget', 'Popularity', 'Release Date', 'Revenue', 'Runtime', 'Vote_average', 'Vote_count']
theMoviesDF_TMDB.to_csv("../DataSources/theMoviesDF_TMDB.csv", index=False, encoding='utf-8')
theMoviesDF_TMDB.head()


,Title,Budget,Popularity,Release Date,Revenue,Runtime,Vote_average,Vote_count
0,Double Indemnity,927262,11.825,1944-04-24,2500000,108,8.2,689
1,Gaslight,2068000,7.708,1944-05-04,0,114,7.4,126
2,Going My Way,0,3.571,1944-05-15,16300000,126,7.0,53
3,Since You Went Away,0,1.283,1944-07-20,0,172,6.3,10
4,Wilson,2995000,1.400,1944-08-01,2000000,154,5.5,6


In [7]:
theMoviesDF_TMDB.shape

(385, 8)

In [19]:
#API Call for 2018 Best Picture Contenders
contenders2018=['Bohemian Rhapsody',
            'Black Panther',
            'BlacKkKlansman',
            'The Favourite',
            'Roma',
            'A Star is Born',
            'Vice',
            'Green Book']

#TMDB API CALLs to get movie id (to be able to make a more specific API call later)
movie_ids_2018 = []
base_url1='https://api.tmdb.org/3/search/movie'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

#for all movies in best picture df, run API call by title and year and extract movie title and movie id
for i in range (len(contenders2018)):
    combined_url1=base_url1+api_key+'&query='+contenders2018[i]+'&year='+str(2018)
    try:
        movieData_TMDB=requests.get(combined_url1).json()
        movie_id=movieData_TMDB['results'][0]['id']
        movie_title=movieData_TMDB['results'][0]['original_title']
        release_date=movieData_TMDB['results'][0]['release_date']
        
        movie_ids_2018.append([movie_title, release_date, movie_id])
    except:
        pass

movie_ids_2018_df=pd.DataFrame(movie_ids_2018)
movie_ids_2018_df.columns=['Title', 'Release Date', 'Movie ID']

#Run API API calls by movie id for more details (2018 contenders)
tmdb_predf=[]
base_url2 = 'https://api.tmdb.org/3/movie/'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

for i in range(len(movie_ids_2018_df)):
    combined_url2=base_url2+ str(movie_ids_2018_df['Movie ID'][i])+ api_key 
    try:
        movieData_TMDB=requests.get(combined_url2).json()
        movie_title=movieData_TMDB['title']
        budget=movieData_TMDB['budget']
        popularity=movieData_TMDB['popularity']
        release_date=movieData_TMDB['release_date']
        revenue=movieData_TMDB['revenue']
        runtime=movieData_TMDB['runtime']
        vote_average=movieData_TMDB['vote_average']
        vote_count=movieData_TMDB['vote_count']
        
        tmdb_predf.append([movie_title, budget, popularity, release_date, revenue, runtime, vote_average, vote_count])
    except:
        pass
    
theMoviesDF2018_TMDB=pd.DataFrame(tmdb_predf)
theMoviesDF2018_TMDB.columns=['Title', 'Budget', 'Popularity', 'Release Date', 'Revenue', 'Runtime', 'Vote_average', 'Vote_count']
theMoviesDF2018_TMDB.to_csv("../DataSources/theMoviesDF2018_TMDB.csv", index=False, encoding='utf-8')
theMoviesDF2018_TMDB



,Title,Budget,Popularity,Release Date,Revenue,Runtime,Vote_average,Vote_count
0,Bohemian Rhapsody,52000000,170.104,2018-10-24,835137710,135,8.2,4895
1,Black Panther,200000000,53.369,2018-02-13,1346739107,134,7.4,10532
2,BlacKkKlansman,15000000,31.767,2018-07-30,48271960,135,7.5,2020
3,The Favourite,15000000,139.806,2018-11-20,44517453,119,7.7,840
4,Roma,15000000,34.205,2018-08-25,0,135,8.0,1086
5,A Star Is Born,36000000,105.901,2018-10-03,418235869,135,7.4,3567
6,Vice,60000000,45.840,2018-12-25,52138055,132,7.2,453
7,Green Book,23000000,56.313,2018-11-16,61904186,130,8.4,965


In [22]:
#API Call for 2016 Best Picture Movies
films2016=['La La Land',
          'Arrival',
          'Lion',
          'Hell or High Water',
          'Hidden Figures',
          'Moonlight',
          'Hacksaw Ridge',
          'Manchester by the Sea',
          'Fences']

#TMDB API CALLs to get movie id (to be able to make a more specific API call later)
movie_ids_2016 = []
base_url1='https://api.tmdb.org/3/search/movie'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

#for all movies in best picture df, run API call by title and year and extract movie title and movie id
for i in range (len(films2016)):
    combined_url1=base_url1+api_key+'&query='+films2016[i]+'&year='+str(2016)
    try:
        movieData_TMDB=requests.get(combined_url1).json()
        movie_id=movieData_TMDB['results'][0]['id']
        movie_title=movieData_TMDB['results'][0]['original_title']
        release_date=movieData_TMDB['results'][0]['release_date']
        
        movie_ids_2016.append([movie_title, release_date, movie_id])
    except:
        pass

movie_ids_2016_df=pd.DataFrame(movie_ids_2016)
movie_ids_2016_df.columns=['Title', 'Release Date', 'Movie ID']

#Run API API calls by movie id for more details (2016)
tmdb_predf=[]
base_url2 = 'https://api.tmdb.org/3/movie/'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

for i in range(len(movie_ids_2016_df)):
    combined_url2=base_url2+ str(movie_ids_2016_df['Movie ID'][i])+ api_key 
    try:
        movieData_TMDB=requests.get(combined_url2).json()
        movie_title=movieData_TMDB['title']
        budget=movieData_TMDB['budget']
        popularity=movieData_TMDB['popularity']
        release_date=movieData_TMDB['release_date']
        revenue=movieData_TMDB['revenue']
        runtime=movieData_TMDB['runtime']
        vote_average=movieData_TMDB['vote_average']
        vote_count=movieData_TMDB['vote_count']
        
        tmdb_predf.append([movie_title, budget, popularity, release_date, revenue, runtime, vote_average, vote_count])
    except:
        pass
    
theMoviesDF2016_TMDB=pd.DataFrame(tmdb_predf)
theMoviesDF2016_TMDB.columns=['Title', 'Budget', 'Popularity', 'Release Date', 'Revenue', 'Runtime', 'Vote_average', 'Vote_count']
theMoviesDF2016_TMDB.to_csv("../DataSources/theMoviesDF2016_TMDB.csv", index=False, encoding='utf-8')
theMoviesDF2016_TMDB

,Title,Budget,Popularity,Release Date,Revenue,Runtime,Vote_average,Vote_count
0,La La Land,30000000,17.561,2016-11-29,445435700,129,7.9,9285
1,Arrival,47000000,20.369,2016-11-10,203388186,116,7.4,9707
2,Lion,12000000,24.197,2016-11-24,140302754,118,8.1,3479
3,Hell or High Water,12000000,12.255,2016-08-12,37589296,102,7.3,2289
4,Hidden Figures,25000000,19.442,2016-12-10,230698791,127,8.0,4360
5,Moonlight,4000000,11.615,2016-10-21,65046687,111,7.3,3551
6,Hacksaw Ridge,40000000,19.047,2016-10-07,175302354,140,8.0,6136
7,Manchester by the Sea,9000000,10.412,2016-11-18,75026965,138,7.5,2899
8,Fences,24000000,10.158,2016-12-16,64414761,139,6.8,1370


In [21]:
#API Call for 2017 Best Picture Movies
films2017=['The Shape of Water',
          'Darkest Hour',
          'Dunkirk',
          'Phantom Thread',
          'The Post',
          'Call Me by Your Name',
          'Lady Bird',
          'Three Billboards Outside Ebbing, Missouri',
          'Get Out']

#TMDB API CALLs to get movie id (to be able to make a more specific API call later)
movie_ids_2017 = []
base_url1='https://api.tmdb.org/3/search/movie'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

#for all movies in best picture df, run API call by title and year and extract movie title and movie id
for i in range (len(films2017)):
    combined_url1=base_url1+api_key+'&query='+films2017[i]+'&year='+str(2017)
    try:
        movieData_TMDB=requests.get(combined_url1).json()
        movie_id=movieData_TMDB['results'][0]['id']
        movie_title=movieData_TMDB['results'][0]['original_title']
        release_date=movieData_TMDB['results'][0]['release_date']
        
        movie_ids_2017.append([movie_title, release_date, movie_id])
    except:
        pass

movie_ids_2017_df=pd.DataFrame(movie_ids_2017)
movie_ids_2017_df.columns=['Title', 'Release Date', 'Movie ID']

#Run API API calls by movie id for more details (2016)
tmdb_predf=[]
base_url2 = 'https://api.tmdb.org/3/movie/'
api_key='?api_key=c0bbf0c3ed0773637721083d25c45ba8'

for i in range(len(movie_ids_2017_df)):
    combined_url2=base_url2+ str(movie_ids_2017_df['Movie ID'][i])+ api_key 
    try:
        movieData_TMDB=requests.get(combined_url2).json()
        movie_title=movieData_TMDB['title']
        budget=movieData_TMDB['budget']
        popularity=movieData_TMDB['popularity']
        release_date=movieData_TMDB['release_date']
        revenue=movieData_TMDB['revenue']
        runtime=movieData_TMDB['runtime']
        vote_average=movieData_TMDB['vote_average']
        vote_count=movieData_TMDB['vote_count']
        
        tmdb_predf.append([movie_title, budget, popularity, release_date, revenue, runtime, vote_average, vote_count])
    except:
        pass
    
theMoviesDF2017_TMDB=pd.DataFrame(tmdb_predf)
theMoviesDF2017_TMDB.columns=['Title', 'Budget', 'Popularity', 'Release Date', 'Revenue', 'Runtime', 'Vote_average', 'Vote_count']
theMoviesDF2017_TMDB.to_csv("../DataSources/theMoviesDF2017_TMDB.csv", index=False, encoding='utf-8')
theMoviesDF2017_TMDB

,Title,Budget,Popularity,Release Date,Revenue,Runtime,Vote_average,Vote_count
0,The Shape of Water,19500000,21.814,2017-12-01,194989655,123,7.3,6513
1,Darkest Hour,30000000,14.713,2017-11-22,147568410,125,7.3,2476
2,Dunkirk,100000000,21.118,2017-07-19,525573161,107,7.4,8442
3,Phantom Thread,35000000,13.060,2017-12-25,46261438,131,7.2,1449
4,The Post,50000000,13.776,2017-12-22,174503458,116,7.0,2243
5,Call Me by Your Name,4000000,17.615,2017-09-01,41063475,132,8.3,4290
6,Lady Bird,10000000,17.003,2017-09-08,76858273,94,7.3,3412
7,"Three Billboards Outside Ebbing, Missouri",15000000,22.585,2017-11-10,158526998,115,8.2,4859
8,Get Out,4500000,24.057,2017-02-24,255017329,104,7.5,7721
